<a href="https://colab.research.google.com/github/prifabiojorge/ProjetodaImersaoDesafio/blob/main/PlataformaFarmaco1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!conda install -c conda-forge rdkit

/bin/bash: line 1: conda: command not found


In [6]:
pip install rdkit-pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 34.4 MB/s eta 0:00:00


In [15]:
pip install pubchempy

  Preparing metadata (setup.py) ... done
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=87e22c0583519f4383e856baa7542a69bb2fbec81fc0f5b50f8f35cd8c661a8f
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy


In [19]:
pip install -q -U google-generativeai


In [35]:
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
from pubchempy import get_compounds
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

def calcular_descritores(smiles):
  """
  Calcula descritores moleculares a partir do SMILES da molécula.

  Args:
    smiles: String SMILES da molécula.

  Returns:
    Um dicionário com os descritores calculados.
  """
  mol = Chem.MolFromSmiles(smiles)
  descritores = {
      'Peso Molecular': Descriptors.MolWt(mol),
      'LogP': Descriptors.MolLogP(mol),
      'Número de Átomos': mol.GetNumAtoms(),
      'Área de Superfície Polar (PSA)': Descriptors.TPSA(mol),
      'Número de Ligações Rotativas': rdMolDescriptors.CalcNumRotatableBonds(mol),
      'Número de Anéis': rdMolDescriptors.CalcNumRings(mol),
      'Número de Doadores de Ligação de Hidrogênio': Descriptors.NumHDonors(mol),
      'Número de Aceptores de Ligação de Hidrogênio': Descriptors.NumHAcceptors(mol),
      'Momento de Dipolo (MR)': Descriptors.MolMR(mol)
  }
  return descritores

# Input do nome do fármaco
nome_farmaco = input("Digite o nome do fármaco: ")

# Busca do SMILES no PubChem
resultados = get_compounds(nome_farmaco, 'name')

if resultados:
  smiles = resultados[0].canonical_smiles  # Pega o primeiro resultado
  print(f"SMILES encontrado: {smiles}")

  # Cálculo dos descritores
  descritores = calcular_descritores(smiles)
  print(f"Descritores da molécula: {descritores}")
else:
  print(f"Nenhum resultado encontrado para '{nome_farmaco}' no PubChem.")

# Formatando os descritores para o prompt
descritores_texto = ', '.join(f'{nome}: {valor:.2f}' for nome, valor in descritores.items())

# Prompt para o Gemini
prompt = f"""
Analise as seguintes propriedades moleculares e me diga se esta molécula pode ser um bom candidato para o tratamento da doença de Chagas, justificando sua resposta:

Propriedades moleculares:
{descritores_texto}

Informações relevantes:
- A doença de Chagas é causada pelo parasita *Trypanosoma cruzi*.
- Alvos biológicos importantes para o *T. cruzi* incluem a enzima cruzaína.
- Biodisponibilidade e baixa toxicidade são desejáveis para candidatos a fármacos.
"""

# Instanciando o modelo Gemini Pro
model = genai.GenerativeModel(model_name="gemini-1.0-pro")

# Requisição ao Gemini
resposta = model.generate_content(contents=[prompt])

# Processamento da resposta (verificar formato da resposta)
print(f"Gemini: {resposta}")

# Extraindo a resposta em Markdown
resposta_markdown = resposta.text

# Imprimindo a resposta formatada
print(f"## Análise do Gemini:\n\n{resposta_markdown}")

Digite o nome do fármaco: benznidazol 
SMILES encontrado: C1=CC=C(C=C1)CNC(=O)CN2C=CN=C2[N+](=O)[O-]
Descritores da molécula: {'Peso Molecular': 260.253, 'LogP': 1.1077, 'Número de Átomos': 19, 'Área de Superfície Polar (PSA)': 90.06, 'Número de Ligações Rotativas': 5, 'Número de Anéis': 2, 'Número de Doadores de Ligação de Hidrogênio': 1, 'Número de Aceptores de Ligação de Hidrogênio': 5, 'Momento de Dipolo (MR)': 67.1701}
Gemini: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': 'Sim, a molécula com as propriedades moleculares fornecidas pode ser um bom candidato para o tratamento da doença de Chagas. Aqui estão as justificativas:\n\n* **Peso molecular (260,25):** O baixo peso molecular pode facilitar a penetração celular e a distribuição no organismo.\n* **LogP (1,11):** Esse valor indica lipofilicidade moderada, o que geralmente é desejável para candidatos a fármacos que precis